#CDIA:::AAI - Blind Search Methods -- Changing the problem

Ibai Laña

In this notebook we will have the previously developed Blind search methods:

*   BFS and BSF_g
*   DFS and DFS_g
*   DLS and DLS_g
*   IDS and IDS_g

and we will try to solve other problems with more complexity than river crossing problem. Now that the algorithms are generalized, this algorithms will be able to solve any problem if we use the same template to define it.


# PROBLEM 8 PUZZLE
We will follow the same approach as with River crossing, trying to define the same 5 features of the problem. Once we have them conceptualized, we will change the class Problem to include them.

## Initialize Problem
The inizialization of the problem includes, given a set of columns and rows (e.g., 3x3), creating a random initial state and a final state that has the numbers ordered.
The actions defined will be just 4, ["Up", "Down", "Left", "Right"], stating the possible movements of the 0 tile.

This will be defined (and processed) in the __ init __ method of the class.

## Is applicable
The logic here is to locate the row and column of the '0', and check if its location is one of the borders (top, down, left or right) or corners of the board to check if a certain action can be done. When 0 is in the center, all actions are available, but in any other case some actions are restricted.

## Effect
For the effect, we only have to locate the "0" and interchange with the number that is in the destination location

## Is final state
Check if a state is the same as final state defined in the inicialization.

## Get Cost and evaluation
This are not applicable for this problem so we will maintain them as for RCproblem





In [ ]:
import numpy as np

In [ ]:
class ProblemP8 ():
    #attributes of the class are empty
    name = ""
    initial_state = {}
    goal_state = {}
    actions = []
    rows = 3
    columns= 3

    def __init__(self): ## init method is the constructor.

        # normally we would go with a random initialization
        numbers= np.arange(self.rows*self.columns)
        np.random.shuffle(numbers)

        #numbers =np.array([1,2,5,3,4,8,0,6,7])##--->> this configuration only requires depth 6 solution
        #numbers =np.array([1,2,5,0,4,8,3,6,7])
        numbers =np.array([1,2,5,3,6,7,0,4,8])
        self.name ="8PuzzleProblem"
        self.initial_state = numbers.reshape(self.rows, self.columns)
        self.goal_state = np.arange(self.rows*self.columns).reshape(self.rows, self.columns)
        self.actions = ["Up", "Down", "Left", "Right"]


    #we need this in case we want other size
    def set_row_cols (self, rows, cols):
        self.rows=rows
        self.columns=cols

    def is_final_state(self, state):
        return np.array_equal(state, self.goal_state)

    #####
    def is_applicable (self, state, action):
        #find where is 0 in the state to know if the change is applicable:
        row = np.where(state==0)[0][0] +1
        col = np.where(state==0)[1][0] +1
        if action == "Up":
          return row!=1
        if action == "Down":
          return row!=self.rows
        if action == "Left":
          return col!=1
        if action == "Right":
          return col!=self.columns


    # this function should exist for all problems, but its internal operation should be changed per problem
    # in this case, the applicability is checked in the expansion function, so the effect of a change is always produced
    def effect (self, state, action):
        row = np.where(state==0)[0][0]
        col = np.where(state==0)[1][0]
        result = state.copy()
        if action == "Up":
            result[row-1,col]=state[row,col]
            result[row,col]=state[row-1,col]

        if action == "Down":
          result[row+1,col]=state[row,col]
          result[row,col]=state[row+1,col]

        if action == "Left":
          result[row,col-1]=state[row,col]
          result[row,col]=state[row,col-1]

        if action == "Right":
          result[row,col+1]=state[row,col]
          result[row,col]=state[row,col+1]

        return result


# we also need cost of an action adn evaluation of the state, for other problems. for River crossing they are defined and return just 1
    def get_cost(self, action, state):
        return 1

    def get_evaluation (self, state):
        return 1


Now we can create objects of the class Problem that will have all of the methods and attributes to use.

In [ ]:
p8problem = ProblemP8()
p8problem.initial_state

array([[1, 2, 5],
       [3, 6, 7],
       [0, 4, 8]])

In [ ]:
print (p8problem.is_applicable(p8problem.initial_state, "Right"))
print (p8problem.is_applicable(p8problem.initial_state, "Left"))
print (p8problem.is_applicable(p8problem.initial_state, "Up"))
print (p8problem.is_applicable(p8problem.initial_state, "Down"))
row = np.where(p8problem.initial_state==0)[0][0]
col = np.where(p8problem.initial_state==0)[1][0]
print ("zero is in col {} row {}".format(col,row))

True
False
True
False
zero is in col 0 row 2


We check the set of actions that lead to the solution from this initial state: r-r-u-u-l-l

They lead us to final state.

In [ ]:
new_state=p8problem.effect(p8problem.initial_state, "Right")
print (new_state)

new_state=p8problem.effect(new_state, "Right")
print (new_state)

new_state=p8problem.effect(new_state, "Up")
print (new_state)

new_state=p8problem.effect(new_state, "Up")
print (new_state)

new_state=p8problem.effect(new_state, "Left")
print (new_state)

new_state=p8problem.effect(new_state, "Left")
print (new_state)

# Expansion Function
The function is similar but it does not include the particular actions of River Crossing problems, but calls to the problem functions, that will be specific for each problem

It receives the problem, with the functions and actions, and a NODE, which includes the following:

* state	(current state)
* parent node (parent node, starts with empty)
* actions (list of actions that led here)
* cost (starting in 0)
* depth (starting in 0)
* evaluation





In [ ]:
# instead of updating the frontier, the expand function only creates a list of child of possible nodes given a current node
# it does not receive states, but NODES, dictionaries that include other information besides state
def expand (node, problem):
    new_nodes = []
    possible_actions = problem.actions
    for action in possible_actions:
        if problem.is_applicable(node["state"], action):
            new_state= problem.effect (node["state"], action)
            new_node = {}
            new_node["state"]=new_state
            new_node["parent_node"]=node
            new_node["actions"]=node["actions"] + [action]
            new_node["cost"]=problem.get_cost(action, new_state)
            new_node["depth"]=node["depth"]+1
            new_node["evaluation"]=problem.get_evaluation(new_state)
            new_nodes.append (new_node)
    return new_nodes


In [ ]:
initial_node={"state":p8problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
nos = expand (initial_node, p8problem)
print (nos)

[{'state': array([[1, 2, 5],
       [0, 6, 7],
       [3, 4, 8]]), 'parent_node': {'state': array([[1, 2, 5],
       [3, 6, 7],
       [0, 4, 8]]), 'parent_node': {}, 'actions': [], 'cost': 0, 'depth': 0, 'evaluation': 1}, 'actions': ['Up'], 'cost': 1, 'depth': 1, 'evaluation': 1}, {'state': array([[1, 2, 5],
       [3, 6, 7],
       [4, 0, 8]]), 'parent_node': {'state': array([[1, 2, 5],
       [3, 6, 7],
       [0, 4, 8]]), 'parent_node': {}, 'actions': [], 'cost': 0, 'depth': 0, 'evaluation': 1}, 'actions': ['Right'], 'cost': 1, 'depth': 1, 'evaluation': 1}]


# BFS - Breadth First Search

```
1. Make a node with the initial problem state
2. Insert node into the frontier data structure
3. WHILE final state not found AND frontier is not empty DO
  3.1 Remove first node from the frontier
  3.2 IF node contains final state THEN final state found
  3.3 IF node doesn’t contain final state THEN
     3.3.1 EXPAND node’s state
     3.3.2 Insert successor nodes into frontier
4. IF final state found THEN
  4.1  RETURN sequence of actions found
5. ELSE  “solution not found”
```

We are maintaing previous coding but generalized to new methods. Besides, we are creating a result dictionary that contains result info so we can use it later


In [ ]:
def BFS(problem):
    # result dictionary
    result = {"method":"BFS", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    # problem = Problem()
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
         # 3.1. get first element of frontier and delete it
        node = frontier[0]


        frontier = frontier[1:]
        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)





In [ ]:
problem=ProblemP8()
print (problem.initial_state)
result = BFS(problem)
print (result)

[[1 2 5]
 [3 6 7]
 [0 4 8]]


KeyboardInterrupt: ignored

# BFS-g: Breadth First Search with Graphs
Same as BFS but with the expanded nodes list

In [ ]:
def BFS_g(problem):
    # result dictionary
    result = {"method":"BFS_g", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

    ####
    expanded = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
         # 3.1. get first element of frontier and delete it
        node = frontier[0]
        frontier = frontier[1:]

        #add to expanded--> we add the state, as the rest of fields will be different
        expanded.append(node["state"])

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            # check if it is expanded before adding to frontier
            if not np.any ([np.all(n["state"]== e) for e in expanded]):
             ##### CHANGE THE CONDITION TO FIT ALL SIZES ARRAYS::
             # if the state of N is not any of the elements in expanded, add to the frontier
            #if n["state"] not in expanded:
                frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

In [ ]:
problem=ProblemP8()
print (problem.initial_state)
result = BFS_g(problem)
print (result)

[[1 2 5]
 [3 6 7]
 [0 4 8]]
{'method': 'BFS_g', 'final_state': {'state': array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]]), 'parent_node': {'state': array([[3, 1, 2],
       [0, 4, 5],
       [6, 7, 8]]), 'parent_node': {'state': array([[3, 1, 2],
       [4, 0, 5],
       [6, 7, 8]]), 'parent_node': {'state': array([[3, 1, 2],
       [4, 7, 5],
       [6, 0, 8]]), 'parent_node': {'state': array([[3, 1, 2],
       [4, 7, 5],
       [0, 6, 8]]), 'parent_node': {'state': array([[3, 1, 2],
       [0, 7, 5],
       [4, 6, 8]]), 'parent_node': {'state': array([[0, 1, 2],
       [3, 7, 5],
       [4, 6, 8]]), 'parent_node': {'state': array([[1, 0, 2],
       [3, 7, 5],
       [4, 6, 8]]), 'parent_node': {'state': array([[1, 2, 0],
       [3, 7, 5],
       [4, 6, 8]]), 'parent_node': {'state': array([[1, 2, 5],
       [3, 7, 0],
       [4, 6, 8]]), 'parent_node': {'state': array([[1, 2, 5],
       [3, 0, 7],
       [4, 6, 8]]), 'parent_node': {'state': array([[1, 2, 5],
       [3, 6, 7],


# DFS: Depth First Search
The only change with respect to BFS is in the step 3.1, where we take out of the frontier the last node

In [ ]:
def DFS(problem):
    # result dictionary
    result = {"method":"DFS", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
         # 3.1. get LAST element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
        print (iterations)
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

In [ ]:
problem=ProblemP8()
problem.initial_state
result = DFS(problem)
print (result)

# DFS_g - Depth First Search with Graph Search
The only change with respect to BFS.gs is in the step 3.1, where we take out of the frontier the last node



In [ ]:
def DFS_g(problem):
    # result dictionary
    result = {"method":"DFS_g", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

    ####
    expanded = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
         # 3.1. get first element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        #add to expanded--> we add the state, as the rest of fields will be different
        expanded.append(node["state"])

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            # check if it is expanded before adding to frontier
            if not np.any ([np.all(n["state"]== e) for e in expanded]):
            # if n["state"] not in expanded:
                frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

In [ ]:
problem=ProblemP8()
problem.initial_state
result = DFS_g(problem)
print (result["final_state"]["actions"])
print ("max frontier: ", result["max_frontier"])
print ("max depth: ", result["max_depth"])
print ("iterations: ", result['iterations'])

# DLS - Depth Limited Search
The only change with respect to DFS is the check for not violating the limits before including a node in the frontier

In [ ]:
def DLS(problem, depth_limit, iteration_limit):
    # result dictionary
    result = {"method":"DLS"+str(depth_limit), "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
         #before doing anything, if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break

         # 3.1. get LAST element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            ##### WE ONLY APPEND IF DEPTH <= LIMIT
            if n["depth"]<=depth_limit:
                frontier.append(n)
            #in other case node is not appended, ending the loop

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

In [ ]:
problem=ProblemP8()
result = DLS(problem, 5, 10000)
print (result)


In [ ]:
result = DLS(problem, 15, 10000)
print (result)

#DLS_g - Depth Limited Search with Graph Search
The only change with respect to DFS_g is the check for not violating the limit before including a node in the frontier

In [ ]:
def DLS_g(problem, depth_limit, iteration_limit):
    # result dictionary
    result = {"method":"DLS_g-"+str(depth_limit), "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

    ####
    expanded = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...
       #before doing anything, if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break
         # 3.1. get first element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        #add to expanded--> we add the state, as the rest of fields will be different
        expanded.append(node["state"])

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            # check if it is expanded before adding to frontier
            if not np.any ([np.all(n["state"]== e) for e in expanded]):
            # if n["state"] not in expanded:
                if n["depth"]<=depth_limit:
                    frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

In [ ]:
problem=ProblemP8()
result = DLS_g(problem, 7, 10000)
print (result)


# IDS - Iterative Deepening Search
The difference with DLS is in the increase of the limit once the frontier get empty, and the increase of the limit.
We will have to change the while condition in order to control the ending of the frontier. Now if the frontier is empty it can be the end of the process or not, depending on the depth

In [ ]:
def IDS(problem, depth_limit, iteration_limit):
    # result dictionary
    result = {"method":"IDS"+str(depth_limit), "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1

    # we add a control of the accepted max depth, starting with 1:
    current_max_depth=1

    #change the loop condition, now we have to control the end of the loop inside it
    while True:
        #if the len is 0, we have reached to the end of the depth. We start again with an increased max depth
        if len (frontier)==0:
            if current_max_depth<depth_limit:
                current_max_depth+=1
                node ={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
                frontier.append(node)
            else:
                result["status"]= "No nodes in the frontier. No solution possible."
                break

        #control if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break

         # 3.1. get LAST element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            ##### WE ONLY APPEND IF DEPTH <= CURRENT MAX DEPTH
            if n["depth"]<=current_max_depth:
                frontier.append(n)
            #in other case node is not appended, ending the loop

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

In [ ]:
problem=ProblemP8()
result = IDS(problem, 8, 1300)
print (result)

# IDS_g - Iterative Deepening Search with Graph Search

The difference with DLS_g is in the increase of the limit once the frontier get empty, and the increase of the limit.

In [ ]:
def IDS_g(problem, depth_limit, iteration_limit):
    # result dictionary
    result = {"method":"IDS_g-"+str(depth_limit), "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
    frontier = []

    ####
    expanded = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
     # we add a control of the accepted max depth, starting with 1:
    current_max_depth=1

    #change the loop condition, now we have to control the end of the loop inside it
    while True:
        #if the len is 0, we have reached to the end of the depth. We start again with an increased max depth
        if len (frontier)==0:
            if current_max_depth<depth_limit:
                current_max_depth+=1
                print ("Exploring depth: ", current_max_depth)
                node ={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":1}
                frontier.append(node)
                expanded = []
            else:
                result["status"]= "No nodes in the frontier. No solution possible."
                break
       #before doing anything, if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break
         # 3.1. get first element of frontier and delete it
        node = frontier[-1]
        frontier = frontier[:-1]

        #add to expanded--> we add the state, as the rest of fields will be different
        expanded.append(node["state"])

        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier
        new_nodes = expand(node, problem)
        for n in new_nodes:
            # check if it is expanded before adding to frontier
            if not np.any ([np.all(n["state"]== e) for e in expanded]):
            # if n["state"] not in expanded:
                if n["depth"]<=current_max_depth:
                    frontier.append(n)

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

In [ ]:
problem=ProblemP8()
result = IDS_g(problem, 28, 30000)
print (result)

# PROBLEM SUDOKU
We will follow the same approach as with prior problems, trying to define the same 5 features of the problem. Once we have them conceptualized, we will change the class Problem to include them.

## Initialize Problem
The problem will be a matrix, with a '0' denoting empty spaces. The action will be to look for the first empty cell and try to put a number (1, 2, 3, ... 9)

The actions will consist of putting numbers in empty spaces.

This will be defined (and processed) in the __ init __ method of the class.

## Is applicable
The function will locate the first '0', then, it will check the desired number to include is not in the same row, column or square

## Effect
Just to change the value in the selected position

## Is final state
There are no places with value 0.

## Get Cost and evaluation
This are not applicable for this problem so we will maintain them as for RCproblem
